In [962]:
import geopandas as gp
import numpy as np
import pandas as pd
from glob import glob
import datetime
import os
import shutil

import pprint
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.39 ms (started: 2022-03-23 15:37:45 -04:00)


# COPY geojson.zips (parcels) files to /samples

In [5]:
lenAddr = len('_addresses')
for file in files:
    namestem = os.path.basename(file).split('.')[0][:-lenAddr]
    shutil.copy(f"/home/nhat/regrid-bucket/{namestem}.geojson.zip", '/home/nhat/mdu/samples/')
    print(namestem)

ca_los_angeles
fl_liberty
ia_adams
ma_middlesex
mo_caldwell
mo_clinton
mo_dekalb
mo_gentry
ny_kings
ny_new_york
tx_loving
time: 26.9 s (started: 2022-02-05 14:16:23 -05:00)


# download samples dir to local

In [ ]:
ls -1 ~/mdu/samples/*.geojson.zip | cat > mdu/download.txt

gcloud compute scp nhat@regrid-tiling-dev:~/mdu/download.txt ~/Downloads/mdu/

parallel --verbose "gcloud compute scp nhat@regrid-tiling-dev:{} ~/Downloads/mdu/parcel/" :::: ~/Downloads/mdu/download.txt



# high rises / MDU

In [3]:
address_fields = ['ll_uuid', 'a_address', 'a_dpv_type', 'a_lat', 'a_lon']
parcel_fields = ['ll_uuid', 'll_bldg_count', 'lbcs_structure', 'lbcs_function', 'address', 'rdi', 'dpv_status', 'dpv_type', 'geometry']

time: 417 µs (started: 2022-02-10 15:14:57 -05:00)


In [ ]:
### CA
#### # TOO LARGE = out of RAM = hang!
#### need: ijson to stream parsing the json

### FL, NY, MO

In [4]:
state = 'ny'

dfP = []
for path in glob(f"samples/{state}*.json"):
    dfP.append(gp.read_file(path))
dfP = pd.concat(dfP)

dfA = []
for path in glob(f"samples/{state}*.csv"):
    dfA.append(gp.read_file(path))
dfA = pd.concat(dfA)


S    227872
H     76983
Name: dpv_type, dtype: int64
H    1640032
S     270833
P      54386
        1378
Name: a_dpv_type, dtype: int64
time: 6min 2s (started: 2022-02-10 15:15:00 -05:00)


In [31]:
# mo: 38 H parcels, 1861 H addresses ; 5893 P (post office box) addresses
# ny: 76983 H parcels; 1640032 H addresses ; 54386 P (post office box)  addresses

time: 221 µs (started: 2022-02-05 15:44:12 -05:00)


In [31]:
dfAcopy.shape

(1966629, 35)

time: 1.72 ms (started: 2022-02-10 17:43:32 -05:00)


In [6]:
dfAcopy = dfA.copy()
dfPcopy = dfP.copy()


time: 1.65 s (started: 2022-02-10 15:21:07 -05:00)


In [45]:
dfA = dfAcopy.copy()
dfP = dfPcopy.copy()


time: 1.52 s (started: 2022-02-10 17:46:50 -05:00)


In [46]:
dfP = dfP[parcel_fields]
dfA = dfA[address_fields]


dfP = dfP.loc[dfP.dpv_type == 'H']
# only keep matched 2nd to highrise parcels
dfA = dfA.loc[dfA.ll_uuid.isin(dfP.ll_uuid), :]

time: 1.4 s (started: 2022-02-10 17:46:59 -05:00)


In [56]:
len(set(dfP.ll_uuid) - set(dfA.ll_uuid))

13718

time: 109 ms (started: 2022-02-10 17:58:33 -05:00)


In [47]:
dfP.shape, dfA.shape # ((76983, 9), (1226483, 5))

((76983, 9), (1226483, 5))

time: 1.54 ms (started: 2022-02-10 17:47:04 -05:00)


In [48]:
dfA.a_dpv_type.value_counts()

H    1218051
S       8210
         222
Name: a_dpv_type, dtype: int64

time: 39.3 ms (started: 2022-02-10 17:47:07 -05:00)


In [49]:
group = dfA.groupby('ll_uuid').size()
group.describe()

count    63265.000000
mean        19.386438
std         65.761541
min          1.000000
25%          4.000000
50%          7.000000
75%         16.000000
max       8801.000000
dtype: float64

time: 178 ms (started: 2022-02-10 17:47:20 -05:00)


In [10]:
#
dfA.ll_uuid = dfA.ll_uuid.fillna("")
dfP.ll_uuid = dfP.ll_uuid.fillna("")
# just last 3 chars of ll_uuid should be enough for visual matching
dfA.ll_uuid = dfA.ll_uuid.str[-4:-1]
dfP.ll_uuid = dfP.ll_uuid.str[-4:-1]

time: 555 ms (started: 2022-02-10 15:22:08 -05:00)


In [11]:
dfA.to_csv(f'kepler/{state}_a.csv', index=False)
dfP.to_csv(f'kepler/{state}_p.csv', index=False)


time: 3.95 s (started: 2022-02-10 15:22:08 -05:00)


# SafeGraph POIs

In [19]:
stats = pd.read_csv('ready_summary_stats.csv')
stats.shape
# FILL RATE overall for open_hours ~ 40%
np.average(stats.OPEN_FILL_RATE, weights=stats.TOTAL_POIS)
# ~ 62%
np.average(stats.PHONE_FILL_RATE, weights=stats.TOTAL_POIS)
# 1.44%
np.average(stats.OPENED_ON_PERCENT, weights=stats.TOTAL_POIS)
# 9.87%
np.average(stats.CLOSED_ON_PERCENT, weights=stats.TOTAL_POIS)

0.0987502800044917

time: 5.57 ms (started: 2022-03-04 13:36:14 -05:00)


In [11]:
files = !ls safegraph/*poi*.csv

dfs = []
for file in files:
    df = pd.read_csv(file)[['location_name',
'brands',
'top_category',
'sub_category',
'naics_code',
'latitude',
'longitude',
'street_address',
'city',
'region',
'postal_code',
'phone_number',
'open_hours',
'category_tags',]]
    dfs.append(df)

dfs = pd.concat(dfs)
    

In [16]:
dfs.shape

(5333501, 14)

time: 1.68 ms (started: 2022-03-14 16:42:49 -04:00)


In [38]:
safegraph_mo = dfs[dfs.region == 'MO']
safegraph_mo.shape

(100554, 14)

time: 276 ms (started: 2022-03-15 00:27:22 -04:00)


In [36]:
safegraph_mo.to_csv('safegraph_MO_POI.csv')

time: 701 ms (started: 2022-03-15 00:25:35 -04:00)


# COUNTIES shp

In [24]:
counties = gp.read_file('https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip')

time: 4.37 s (started: 2022-03-14 17:18:48 -04:00)


In [41]:
mo_shp = counties[counties.STATEFP == '29'][['GEOID', 'geometry',]]
mo_shp.shape

(115, 2)

time: 5.03 ms (started: 2022-03-15 00:28:34 -04:00)


In [42]:
mo_shp.to_file('mo_counties.geojson', driver='GeoJSON')

time: 1.08 s (started: 2022-03-15 00:28:35 -04:00)


In [45]:
counties3 = mo_shp[mo_shp.GEOID.isin(['29049',
'29147',
'29227',])]
counties3

,GEOID,geometry
1575,29227,"POLYGON ((-94.46066 40.57110, -94.46047 40.571..."
2232,29147,"POLYGON ((-95.17904 40.31964, -95.17904 40.319..."
2846,29049,"POLYGON ((-94.60224 39.64563, -94.60224 39.645..."


time: 24.4 ms (started: 2022-03-15 00:34:49 -04:00)


## Counts: Safegraph v.s. ATTOM

In [69]:
safegraph_mo = gp.GeoDataFrame(
    safegraph_mo, geometry=gp.points_from_xy(safegraph_mo.longitude, safegraph_mo.latitude)).set_crs(counties3.crs)
safegraph_mo.shape

(100554, 15)

time: 32.2 ms (started: 2022-03-15 00:44:24 -04:00)


In [70]:
attom_poi = pd.read_csv('attom/POI_sample.csv')
attom_poi = gp.GeoDataFrame(
    attom_poi, geometry=gp.points_from_xy(attom_poi.LONGITUDE, attom_poi.LATITUDE)).set_crs(counties3.crs)
attom_poi.shape

(1929, 21)

time: 12 ms (started: 2022-03-15 00:44:28 -04:00)


In [71]:
safegraph_counties3 = gp.sjoin(counties3, safegraph_mo, how = 'left', predicate = 'contains')
safegraph_counties3.shape

(737, 17)

time: 154 ms (started: 2022-03-15 00:44:32 -04:00)


In [72]:
attom_counties3 = gp.sjoin(counties3, attom_poi, how = 'left', predicate = 'contains')
attom_counties3 .shape

(1875, 23)

time: 16 ms (started: 2022-03-15 00:44:35 -04:00)


# DEMAND POINTS: ATTOM vs REGRID

In [1065]:
attom_cols = [
    '[ATTOM ID]', 
'SitusCounty',
'SitusStateCountyFIPS',
 'AreaBuilding',
 'AreaBuildingDefinitionCode',
 'AreaLotAcres',
 'AreaLotSF',
 'BedroomsCount',
 'BuildingsCount',
 'CensusBlock',
 'CompanyFlag',
 'ContactOwnerMailAddressFull',
 'ContactOwnerMailAddressInfoFormat',
 'GeoQuality',
 'LegalBlock1',
 'LegalDescription',
 'LegalLotNumber1',
 'LegalLotNumber2',
 'LegalQuarter',
 'LegalQuarterQuarter',
 'LegalRange',
 'LegalSection',
 'LegalSubdivision',
 'LegalTownship',
 'LegalTractNumber',
 'LegalUnit',
 'OwnerTypeDescription1',
 'OwnerTypeDescription2',
 'OwnershipVestingRelationCode',
 'ParcelAccountNumber',
 # 'ParcelNumberAlternate',
 'ParcelNumberPrevious',
 'ParcelNumberRaw',
 'ParcelNumberYearAdded',
 'ParcelNumberYearChange',
 'PartyOwner1NameFirst',
 'PartyOwner1NameFull',
 'PartyOwner1NameLast',
 'PartyOwner1NameMiddle',
 'PartyOwner1NameSuffix',
 'PartyOwner2NameFirst',
 'PartyOwner2NameFull',
 'PartyOwner2NameLast',
 'PartyOwner2NameMiddle',
 'PartyOwner2NameSuffix',
 'PartyOwner3NameFirst',
 'PartyOwner3NameFull',
 'PartyOwner3NameLast',
 'PartyOwner3NameMiddle',
 'PartyOwner3NameSuffix',
 'PropertyAddressCity',
 'PropertyAddressFull',
 'PropertyAddressHouseNumber',
 'PropertyAddressState',
 'PropertyAddressStreetDirection',
 'PropertyAddressStreetName',
 'PropertyAddressStreetPostDirection',
 'PropertyAddressStreetSuffix',
 'PropertyAddressUnitPrefix',
 'PropertyAddressUnitValue',
 'PropertyAddressZIP',
 'PropertyAddressZIP4',
 'PropertyLatitude',
 'PropertyLongitude',
 'PropertyUseGroup',
 'PropertyUseStandardized',
 'StatusOwnerOccupiedFlag',
 'StoriesCount',
 'TopographyCode',
 'UnitsCount',
 'UtilityBuildingArea',
 'UtilityBuildingFlag',
 'YearBuilt',
 'YearBuiltEffective',
     ]

len(attom_cols)

73

time: 3.12 ms (started: 2022-03-24 17:16:41 -04:00)


In [1391]:
attom = pd.read_csv('attom/AS/AS-sample.csv')[attom_cols]
attom = gp.GeoDataFrame(
    attom, geometry=gp.points_from_xy(attom.PropertyLongitude, attom.PropertyLatitude)).set_crs(counties3.crs)
print(attom.shape) # (32312, 75)
attom = gp.sjoin(attom, counties3, how = 'left', predicate = 'within')

# drop ATTOM points that lie outside county boundary
attom = attom.dropna(axis = 0, subset = ['GEOID']).drop(
    columns=['index_right', 'SitusStateCountyFIPS']).rename(columns={'GEOID' : 'COUNTY_FIPS'})

print(attom.shape) # (32050, 75)

# attom use 2010[?] county shp
# attom['SitusStateCountyFIPS'] = attom.SitusStateCountyFIPS.astype(str).str.zfill(5)
# attom['geoid_diff'] = (attom.SitusStateCountyFIPS == attom.GEOID)
# attom[attom.geoid_diff == False].to_csv('attom_mixed_points.csv', index=None)
# attom['geoid_diff'].value_counts()

(32312, 74)
(32050, 74)
time: 860 ms (started: 2022-03-28 11:56:22 -04:00)


In [ ]:
# ParcelAccountNumber coverage = 48% nationwide, but all=NaN in the provided samples 
# FUTURE QUESTION: what does ParcelAccountNumber look like?
attom.ParcelAccountNumber.isnull().sum()

In [297]:
regrid_cols = [
 'address',
 'address_source',
 'book',
 'cass_errorno',
 'census_block',
 # 'census_blockgroup',
 # 'census_tract',
 'city',
 'county',
 # 'deedacres',
 'dpv_codes',
 'dpv_notes',
 'dpv_status',
 'dpv_type',
 'geoid',
 'geometry',
 'gisacre',
 'lat',
 'lbcs_activity',
 'lbcs_activity_desc',
 'lbcs_function',
 'lbcs_function_desc',
 'lbcs_ownership',
 'lbcs_ownership_desc',
 'lbcs_site',
 'lbcs_site_desc',
 'lbcs_structure',
 'lbcs_structure_desc',
 'legaldesc',
 'll_bldg_count',
 'll_bldg_footprint_sqft',
 'll_gisacre',
 'll_gissqft',
 'll_updated_at',
 'll_uuid',
 'lon',
 'mail_city',
 'mail_state2',
 'mail_zip',
 'mailadd',
 'original_address',
 'owner',
 'owner2',
 'page',
 'parcelnumb',
 # 'path',
 'rdi',
 'saddno',
 'saddpref',
 'saddstr',
 'saddstsuf',
 'saddsttyp',
 'saledate',
    'saleprice',
 'scity',
 # 'sec',
 # 'sec_twp_rn',
 # 'section',
 'sunit',
 'szip',
 'usps_vacancy',
 'usps_vacancy_date']

time: 685 µs (started: 2022-03-17 23:14:42 -04:00)


### [LATER] ll_address_count: A count of all the addresses we know of existing on a parcel. This includes the primary situs address and all the secondary or unit based addresses in our matched secondary address dataset. It is similar to the county provided 'numunits' in our schema, but calculated by us based on our nationwide address dataset from Ageon.	"Expected April 2022"

In [1067]:
regrid_files = [ '/home/nhat/regrid-bucket/mo_worth.geojson.zip',
    '/home/nhat/regrid-bucket/mo_clinton.geojson.zip',
    '/home/nhat/regrid-bucket/mo_nodaway.geojson.zip',]

regrid = []
for file in regrid_files:
    df = gp.read_file(file)
    regrid.append(df)
# counties3.crs # EPSG:4269
# regrid.crs # EPSG:4326
regrid = pd.concat(regrid)

regrid['polygon'] = regrid.geometry.to_crs(counties3.crs)
# regrid = regrid[regrid_cols]

# Change the geometry column
regrid = gp.GeoDataFrame(regrid, geometry =
                         gp.points_from_xy(regrid.lon, regrid.lat)).set_crs(counties3.crs)
print(regrid.shape) 

regrid[['lat', 'lon']].isnull().sum()

(27010, 136)


lat    0
lon    0
dtype: int64

time: 14.6 s (started: 2022-03-24 17:17:50 -04:00)


In [1068]:
# Some points are dropped -- QUESTION: which TIGER county shp year does REGRID use?
regrid = gp.sjoin(regrid, counties3, how = 'left', predicate = 'within')
print(regrid.shape,  regrid.dropna(axis = 0, subset = ['GEOID']).shape) 
regrid = regrid.dropna(axis = 0, subset = ['GEOID'])

regrid['geoid_diff'] = (regrid.geoid == regrid.GEOID)
print("REGRID has no geoid_diff", regrid['geoid_diff'].value_counts())

regrid = regrid.drop(columns=['geoid', 'index_right', 'geoid_diff']).rename(columns= {'GEOID': 'COUNTY_FIPS'})


(27010, 138) (27003, 138)
REGRID has no geoid_diff True    27003
Name: geoid_diff, dtype: int64
time: 715 ms (started: 2022-03-24 17:18:12 -04:00)


In [1069]:
# Some null Parcel ID/Parcel Number
regrid[['parcelnumb']].isnull().sum()

parcelnumb    11
dtype: int64

time: 8.07 ms (started: 2022-03-24 17:18:23 -04:00)


# Pre-processing

In [1070]:
# Formatting
cols = ['legaldesc', 'parcelnumb']
for col in cols:
    regrid[col] = regrid[col].str.strip()
    regrid[col] = regrid[col].str.upper()
regrid[cols] = pd.DataFrame(
    np.core.defchararray.replace(regrid[cols].values.astype(str), ' ', '-'), 
    regrid.index, cols)


cols = ['LegalDescription', 'ParcelNumberRaw']
for col in cols:
    attom[col] = attom[col].str.strip()
    attom[col] = attom[col].str.upper()
attom[cols] = pd.DataFrame(
    np.core.defchararray.replace(attom[cols].values.astype(str), ' ', '-'), 
    attom.index, cols)

# Coords should have float types
regrid[['lat', 'lon',]] = regrid[['lat', 'lon',]].astype(float)
attom[['PropertyLatitude', 'PropertyLongitude',]] = attom[['PropertyLatitude', 'PropertyLongitude', ]].astype(float)

attom = attom.rename(columns={'[ATTOM ID]': 'at_id'})
regrid = regrid.rename(columns={'ll_uuid': 're_id'})
attom.shape, regrid.shape

((32050, 74), (27003, 136))

time: 598 ms (started: 2022-03-24 17:22:49 -04:00)


# Merge on parcelNumberRaw / parcelnumb

In [292]:
# parcelnumb has 99% coverage
print(regrid.shape, regrid.dropna(subset=['parcelnumb']).shape)
regrid = regrid.dropna(subset=['parcelnumb'])

(27003, 136) (26992, 136)
time: 122 ms (started: 2022-03-17 23:08:57 -04:00)


In [293]:
# 'ParcelNumberAlternate' is useless
# ParcelNumberRaw should have 100% coverage
print(attom.shape, attom.dropna(subset=['ParcelNumberRaw']).shape)
attom = attom.dropna(subset = ['ParcelNumberRaw'])

(32050, 75) (32050, 75)
time: 46.6 ms (started: 2022-03-17 23:09:00 -04:00)


In [356]:
cols = attom_cols + regrid_cols + ['COUNTY_FIPS']
cols = sorted(set(cols) - set(['SitusStateCountyFIPS', 'geoid', 'geometry']))
len(cols)

129

time: 1.94 ms (started: 2022-03-17 23:39:47 -04:00)


In [367]:
# Only 6% of parcels have matching parcel number
parcel_num_match = attom.merge(regrid, left_on = 'ParcelNumberRaw', 
                               right_on = 'parcelnumb', how = 'inner')

# rows with matching parcel number also has IDENTICAL COUNTY_FIPS
parcel_num_match[['COUNTY_FIPS_x', 'COUNTY_FIPS_y']].astype(int).diff(axis=1).COUNTY_FIPS_y.unique()

parcel_num_match = parcel_num_match.rename(columns={'COUNTY_FIPS_x': 'COUNTY_FIPS'}).drop(columns='COUNTY_FIPS_y')[cols]
parcel_num_match.shape

(2000, 129)

time: 111 ms (started: 2022-03-17 23:47:41 -04:00)


In [336]:
x = ['123'
'456']
print(x)
x

['123456']


['123456']

time: 2.17 ms (started: 2022-03-17 23:33:53 -04:00)


# YES! legal_matched most of ATTOM parcels to REGRID parcels!

In [389]:
set(attom.COUNTY_FIPS), set(regrid.COUNTY_FIPS), 

({'29049', '29147', '29227'}, {'29049', '29147', '29227'})

time: 4.86 ms (started: 2022-03-18 00:05:15 -04:00)


In [408]:
print(attom.shape, attom.dropna(subset=['LegalDescription']).shape)
attom_legal = attom.dropna(subset=['LegalDescription'])
print(regrid.shape, regrid.dropna(subset=['legaldesc']).shape)
regrid_legal = regrid.dropna(subset=['legaldesc'])


(32050, 75) (32025, 75)
(26992, 136) (26852, 136)
time: 192 ms (started: 2022-03-18 00:19:49 -04:00)


In [435]:
top_legal = ['SESE', 'SWSW', 'NWNW', 'NENE', 'SE1/4']
attom[test_cols].groupby('LegalDescription').size().sort_values(ascending=False).head(5).keys()

Index(['SESE', 'SWSW', 'NWNW', 'NENE', 'SE1/4'], dtype='object', name='LegalDescription')

time: 33.5 ms (started: 2022-03-18 12:25:05 -04:00)


In [476]:
# ATTOM has unique parcel number; and most legaldesc are unique
attom.shape, attom.ParcelNumberRaw.unique().shape, attom.LegalDescription.unique().shape

((32050, 75), (32050,), (31143,))

time: 12.7 ms (started: 2022-03-18 12:46:03 -04:00)


In [789]:
# Many ATTOM ParcelNumberRaw contains many whitespaces; change ATTOM whitespace into REGRID hyphen 
attom['ParcelNumberRaw'] = attom.ParcelNumberRaw.replace({' ' : '-'}, regex=True)
# NOTE: Some REGRID parcelnumb contains single whitespaces
regrid['parcelnumb'] = regrid.parcelnumb.replace({' ' : '-'}, regex=True)


time: 49.5 ms (started: 2022-03-22 16:02:41 -04:00)


In [477]:
# REGRID has bad/non-unique parcel numbers; most legaldesc are unique
regrid.shape, regrid.parcelnumb.count(), regrid.parcelnumb.unique().shape, regrid.legaldesc.unique().shape, 

((26992, 136), 26992, (15704,), (26111,))

time: 15.6 ms (started: 2022-03-18 12:46:14 -04:00)


In [ ]:
test_cols = [
    'parcelnumb',
    'lat', 'lon',
    'address',
     'legaldesc', 
      'owner',
]

by = 'legaldesc'
top_legal_regrid = regrid.groupby(by).size().sort_values(ascending=False).head(5).index.to_list()

for k, v in regrid[test_cols].groupby(by):
    if k in top_legal_regrid[3:]:
        print("******* CURRENT KEY IS ", k)
        print(v.parcelnumb.to_list())
        v = v.sort_values(by=['parcelnumb',]).drop(by, axis=1)
        print(v.to_string(index=False))

******* CURRENT KEY IS  SE1/4
['09-08.2-33-000-000-007.000', '07-01.1-12-000-000-005.000', '06-07.0-25-000-000-004.000', '05-01.2-11-000-000-009.000', '15-04.2-18-000-000-005.000', '15-01.2-02-000-000-003.000', '09-05.1-15-000-000-006.000', '09-03.0-07-000-000-010.000', '07-09.1-32-000-000-004.000', '06-01.0-02-000-000-007.000', '07-04.1-20-000-000-005.000', '08-03.0-07-000-000-005.000', '06-08.2-28-000-000-004.000', '02-09.2-31-000-000-005.000', '07-04.2-18-000-000-005.000', '07-02.0-03-000-000-006.000', '03-08.1-27-000-000-010.000', '03-07.0-25-000-000-005.000', '07-08.0-34-000-000-005.000', '02-09.1-32-000-000-005.000', '14-06.0-23-000-000-008.000']
                parcelnumb       lat        lon                            address                                      owner
02-09.1-32-000-000-005.000 39.707273 -94.398312                          MO HWY 33                 CROSS AARON O & JENNAFER J
02-09.2-31-000-000-005.000 39.707367 -94.417327                        NW 336TH ST     

In [491]:
attom_test_cols = [
    'LegalDescription',
  'ParcelNumberRaw',
 'PartyOwner1NameFull',
  'PropertyAddressFull',
  'PropertyLatitude',
     'PropertyLongitude',
]

for k,v in attom[attom_test_cols].groupby('LegalDescription'):
    if k in top_legal:
        print("******* CURRENT KEY IS ", k)
        v = v.sort_values(by=['ParcelNumberRaw',]).drop('LegalDescription', axis=1)
        print(v.to_string(index=False))


******* CURRENT KEY IS  NENE
           ParcelNumberRaw                                PartyOwner1NameFull        PropertyAddressFull  PropertyLatitude  PropertyLongitude
02 07.1 36 000 000 001.000                                    GROEBE LAND LLC      10842 NE GILCHRIST RD         39.716348         -94.320739
03 08.1 27 000 000 001.000                                            ATH LLC            OFF NW 352ND ST         39.731047         -94.472547
05 06.2 23 000 000 001.000                            MARY QUEEN BENEDICTINES                NW 310TH ST         39.658141         -94.565400
05 08.0 34 000 000 001.000       RINGER JAMES A & ANN S JOINT DECLAR OFUST TR               NW KEITHS RD         39.629276         -94.584208
06 04.0 17 000 000 001.000                                      DALE F BORTON                 NW MOSS LN         39.672456         -94.510437
06 04.0 18 000 000 001.000                                       BRUCE D MOSS                   NW HWY K         39.672

In [626]:
attom_legal_group = attom.groupby('LegalDescription').size().sort_values(ascending=False)
width = 30
attom_legal_group.index = attom_legal_group.index.str.pad(width, side='right', fillchar=' ').str[:width]
attom_legal_group

LegalDescription
SESE                              31
SWSW                              30
NWNW                              29
NENE                              28
SE1/4                             21
                                  ..
COM NW COR 11-64-35 TH S 1700(     1
COM NW COR 10-65-34 TH E 29.50     1
COM NW COR 1-64-38 TH E 1220 S     1
COM NW COR 06-63-36, S 68 RODS     1
YEAMANS ADD. LOT 5 & 6 EXC. 6      1
Length: 31142, dtype: int64

time: 51.4 ms (started: 2022-03-18 15:26:24 -04:00)


In [627]:
regrid_legal_group = regrid.groupby('legaldesc').size().sort_values(ascending=False)
width = 30
regrid_legal_group.index = regrid_legal_group.index.str.pad(width, side='right', fillchar=' ').str[:width]
regrid_legal_group


legaldesc
SESE                              30
NWNW                              28
NENE                              28
SWSW                              28
SE1/4                             21
                                  ..
COM 660' N & 60 W OF SE COR 7-     1
COM 660 S NE COR SW 1/4 SEC 16     1
COM 66' W SW COR BLK 24 COMSTO     1
COM 66' W SW COR BLK 24 COMSTO     1
tract of land beg @ 295'' e of     1
Length: 26110, dtype: int64

time: 48.4 ms (started: 2022-03-18 15:26:45 -04:00)


In [392]:
# REGRID vs ATTOM : TIGER county FIPS

diff_county = legal_matched[['COUNTY_FIPS_x', 'COUNTY_FIPS_y']].astype(int).diff(axis=1)
diff_county_rows = legal_matched[diff_county.COUNTY_FIPS_y != 0]
diff_county_rows.to_csv('legaldesc_matched.csv',index=False)
print(diff_county_rows.shape)
diff_county_rows[[
    # ATTOM
    'COUNTY_FIPS_x', 'PropertyLatitude', 'PropertyLongitude', 
    # REGRID                                           
    'COUNTY_FIPS_y', 'lat', 'lon',
]]


(3356, 211)


,COUNTY_FIPS_x,PropertyLatitude,PropertyLongitude,COUNTY_FIPS_y,lat,lon
58,29147,40.271797,-95.057220,29227,40.555659,-94.562600
59,29147,40.271797,-95.057220,29227,40.477510,-94.413707
60,29147,40.271797,-95.057220,29227,40.429702,-94.459644
61,29147,40.271797,-95.057220,29227,40.494593,-94.412517
62,29147,40.271797,-95.057220,29227,40.491202,-94.285050
...,...,...,...,...,...,...
13984,29227,40.459210,-94.273690,29049,39.485549,-94.360993
13985,29049,39.499824,-94.474940,29227,40.459102,-94.273742
15661,29049,39.497443,-94.413605,29227,40.428801,-94.346594
15664,29227,40.428823,-94.346473,29049,39.497443,-94.413605


time: 22.4 ms (started: 2022-03-18 00:08:03 -04:00)


In [383]:
# TODO HERE NOW: rows with matching legal description sometimes are matched to different COUNTIES?
legal_matched[['COUNTY_FIPS_x', 'COUNTY_FIPS_y']].astype(int).diff(axis=1).COUNTY_FIPS_y.unique()


array([   0,   80,  -98,  178,   98, -178,  -80])

time: 120 ms (started: 2022-03-18 00:02:25 -04:00)


In [309]:
parcel_num_match[['ParcelNumberRaw', 'parcelnumb']].isnull().sum()

ParcelNumberRaw    0
parcelnumb         0
dtype: int64

time: 4.56 ms (started: 2022-03-17 23:18:30 -04:00)


### Matching owner name, or address? No good!

In [594]:
attom[['PropertyAddressFull', 'PartyOwner1NameFull', 'PartyOwner2NameFull']].sample(3)

,PropertyAddressFull,PartyOwner1NameFull,PartyOwner2NameFull
6799,506 W MISSOURI AVE,TRAVIS W FUHR,KIMBERLY D FUHR
14846,NaN,HUNT,KERWIN
14213,NaN,JACK SPAINHOWER,PATTY SPAINHOWER


time: 8.01 ms (started: 2022-03-18 14:06:47 -04:00)


In [592]:
regrid[['address', 'owner', 'owner2']].sample(3)

,address,owner,owner2
1135,5275 SW MCCULLOCH RD,WALTON JOHN W TRUSTEE,JOHN W WALTON TRUST
10364,MEADOWLARK RD,MAY TIMOTHY D & LYNN G,None
9640,NE CROUCH RD,MM LAZY EIGHT LLC,None


time: 7.17 ms (started: 2022-03-18 14:06:25 -04:00)


### Could we match lat+lon? Regrid OK, Attom Bad

In [561]:
regrid['coords'] = regrid.lat + '!@#' + regrid.lon
regrid.groupby('coords').size().sort_values(ascending=False).head()

coords
39.455355!@#-94.239171    1
40.349641!@#-94.674177    1
40.349914!@#-94.865444    1
40.349909!@#-94.865207    1
40.349906!@#-94.871268    1
dtype: int64

time: 32.1 ms (started: 2022-03-18 13:43:42 -04:00)


In [562]:
attom['coords'] = attom.PropertyLatitude.astype(str) + '!@#' + attom.PropertyLongitude.astype(str)
attom.groupby('coords').size().sort_values(ascending=False).head()

coords
40.5215!@#-95.122         278
40.5292!@#-95.0053        248
39.5366!@#-94.3117        248
39.7411!@#-94.2344         96
40.360774!@#-94.883434     64
dtype: int64

time: 64.9 ms (started: 2022-03-18 13:43:46 -04:00)


In [602]:
# Put all 3 matching types together; Duplicate included
latlon_matched = attom.merge(regrid, on='coords', how='inner')
legal_matched = attom.merge(regrid, left_on = 'LegalDescription', right_on = 'legaldesc', how = 'inner')
num_matched = attom.merge(regrid, left_on = 'ParcelNumberRaw', right_on = 'parcelnumb', how = 'inner')
latlon_matched.shape, legal_matched.shape, num_matched.shape

latlon_ids = set(latlon_matched.ll_uuid)
legal_ids = set(legal_matched.ll_uuid)
num_ids = set(num_matched.ll_uuid)
print(len(latlon_ids), len(legal_ids), len(num_ids))
print('latlon vs legal   :', len(latlon_ids - legal_ids), '/', len(legal_ids - latlon_ids),) 
print('latlon vs num     :', len(latlon_ids - num_ids), '/', len(num_ids - latlon_ids), )
print('num    vs legal   :', len(num_ids - legal_ids), '/', len(legal_ids - num_ids), )

((9979, 215), (26845, 216), (14954, 216))

time: 1.74 s (started: 2022-03-18 15:08:20 -04:00)


# STEP 1: Legal desc match

In [1072]:
# IMPORTANT: first, drop the duplicate values
regrid_legal_uniq = regrid.drop_duplicates(subset='legaldesc', keep=False)
attom_legal_uniq = attom.drop_duplicates(subset='LegalDescription', keep=False)

legal_matched = attom_legal_uniq.merge(regrid_legal_uniq, left_on = 'LegalDescription', right_on = 'legaldesc', how = 'inner')
# ((27003, 136), (25858, 136), (32050, 74), (30778, 74), (14140, 210), True)
regrid_legal_uniq.shape, attom_legal_uniq.shape, legal_matched.shape

((25858, 136), (30778, 74), (14140, 210))

time: 588 ms (started: 2022-03-24 17:25:30 -04:00)


## Similarity score

In [1073]:
# NO EMPTY 'parcelnumb', 'ParcelNumberRaw'
legal_matched.shape, legal_matched[['parcelnumb', 'ParcelNumberRaw']].count()

((14140, 210),
 parcelnumb         14140
 ParcelNumberRaw    14140
 dtype: int64)

time: 57.9 ms (started: 2022-03-24 17:25:34 -04:00)


In [1085]:
# NO short ParcelNumberRaw, Some short parcelnumb
legal_matched[legal_matched.ParcelNumberRaw.str.len() < 10].shape, legal_matched[legal_matched.parcelnumb.str.len() < 10].shape

((0, 210), (2856, 210))

time: 20.4 ms (started: 2022-03-24 17:28:53 -04:00)


In [ ]:
legal_matched[['parcelnumb', 'ParcelNumberRaw']].sample(10)

,parcelnumb,ParcelNumberRaw
2873,05000,22-04-19-02-03-05000
5211,01-06.0-23-002-020-006.000,01-06.0-23-002-020-006.000
5914,15-05.0-15-000-000-009.001,15-05.0-15-000-000-009.001
10910,12-06.2-23-000-000-006.001,12-06.2-23-000-000-006.001
12309,07-08-09-31-1.02,07-08-09-31-1.02
4394,13-05.0-15-000-000-005.036,13-05.0-15-000-000-005.036
11593,13000,22-04-18-01-02-13000
974,05-08.0-34-003-001-001.000,05-08.0-34-003-001-001.000
10150,10-07.1-36-001-001-002.001,10-07.1-36-001-001-002.001
2951,25000,22-04-19-02-15-25000


time: 7.08 ms (started: 2022-03-24 17:27:33 -04:00)


#### Short parcelnumb

In [1111]:
match_cols = [
# '[ATTOM ID]', 'll_uuid',
'at_id', 're_id',
'legaldesc', 'LegalDescription',
'parcelnumb', 'ParcelNumberRaw', 
'address', 'PropertyAddressFull', 
'owner', 'PartyOwner1NameFull', 
'owner2', 'PartyOwner2NameFull',
'lat', 'lon',
'PropertyLatitude', 'PropertyLongitude', 
'saddno', 'PropertyAddressHouseNumber',
]

time: 443 µs (started: 2022-03-24 17:50:18 -04:00)


In [1117]:
legal_short_numb = legal_matched[legal_matched.parcelnumb.str.len() < 10][match_cols]
legal_short_numb[['lat', 'lon']] = legal_short_numb[['lat', 'lon']].astype(float)

legal_short_numb['legal_shorted_numberraw'] = legal_short_numb.ParcelNumberRaw.str.split('-').str[-1]
legal_short_numb['legal_num_sameness'] = legal_short_numb.apply(lambda row: row.legal_shorted_numberraw in row.parcelnumb, axis = 1)
legal_short_numb['legal_num_sameness'].value_counts()

True     2854
False       2
Name: legal_num_sameness, dtype: int64

time: 56.8 ms (started: 2022-03-24 17:52:05 -04:00)


#### Full length parcelnumb

In [1118]:
legal_full_numb = legal_matched[legal_matched.parcelnumb.str.len() >= 10][match_cols]
legal_full_numb['legal_num_sameness'] = (legal_full_numb.ParcelNumberRaw == legal_full_numb.parcelnumb)
legal_full_numb['legal_num_sameness'].value_counts()

True     11281
False        3
Name: legal_num_sameness, dtype: int64

time: 45.7 ms (started: 2022-03-24 17:52:07 -04:00)


In [1119]:
short_false = legal_short_numb[legal_short_numb['legal_num_sameness'] == False]
short_true = legal_short_numb[legal_short_numb['legal_num_sameness'] == True]
full_false = legal_full_numb[legal_full_numb['legal_num_sameness'] == False]
full_true = legal_full_numb[legal_full_numb['legal_num_sameness'] == True]

print(short_false.shape, short_true.shape, full_false.shape, full_true.shape)

(2, 20) (2854, 20) (3, 19) (11281, 19)
time: 9.16 ms (started: 2022-03-24 17:52:09 -04:00)


### Manual location check: Kepler-GL
https://docs.kepler.gl/docs/keplergl-jupyter#6-match-config-with-data

#### Config save & load trick

In [1095]:
config_name = 'config_legal'

# LOAD the config  using jupyter %run magic ; Note the parameter escape {}
# i.e. execute variable assignment: config = ....
%run {config_name}.py

legal_map = KeplerGl(height=800, data={
    'short_false' : short_false,
    'short_true' : short_true,
    'full_false' : full_false,
    'full_true' : full_true, 
}, config = my_config)

legal_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'config': {'mapState': {'bearing': 0, 'dragRotate': False, 'isSplit': False, 'latitude': 40.4…

time: 618 ms (started: 2022-03-24 17:35:24 -04:00)


In [964]:
# TODO: ignore/how to resolve? Identified some (legaldesc matched, short parcelnumb matched, but mapping to diff locations)!

time: 296 µs (started: 2022-03-24 11:32:12 -04:00)


In [1121]:
full_true.shape, full_true[['saddno', 'PropertyAddressHouseNumber',]].count(), short_true.shape, short_true[['saddno', 'PropertyAddressHouseNumber',]].count()

((11281, 19),
 saddno                        8031
 PropertyAddressHouseNumber    7050
 dtype: int64,
 (2854, 20),
 saddno                           0
 PropertyAddressHouseNumber    2360
 dtype: int64)

time: 8.67 ms (started: 2022-03-24 17:53:17 -04:00)


In [1125]:
full_true.shape, full_true[~(full_true.saddno.isna()) | ~(full_true.PropertyAddressHouseNumber.isna())].shape, full_true[~(full_true.saddno.isna()) & ~(full_true.PropertyAddressHouseNumber.isna())].shape, 

((11281, 19), (8374, 19), (6707, 19))

time: 11.8 ms (started: 2022-03-24 17:56:42 -04:00)


In [1129]:
config_name = 'config_legal'

# LOAD the config  using jupyter %run magic ; Note the parameter escape {}
# i.e. execute variable assignment: config = ....
%run {config_name}.py

legal_map = KeplerGl(height=600, data={
    'short_false' : short_false,
    'short_true' : short_true1,
    'full_false' : full_false,
    'full_true' : full_true1, 
}, config = my_config)

legal_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'config': {'mapState': {'bearing': 0, 'dragRotate': False, 'isSplit': False, 'latitude': 40.4…

time: 282 ms (started: 2022-03-24 17:58:20 -04:00)


In [ ]:
# Sole error:
# parcelnumb
# 03-09.0-29-000-000-003.000
# ParcelNumberRaw
# 11-09.0-29-000-000-001.000
# address
# NW 348TH ST/ NW WATSON RD
# owner
# JOHNSON CHARLOTTE A TRUSTEE
# owner2
# PropertyAddressFull
# SW HORD RD
# PartyOwner1NameFull
# BOYLE RICHARD JAMES TRUST
# PartyOwner2NameFull
# nan
# legaldesc
# SWNW-SEC-29
# PropertyLatitude
# 39.555405
# lat
# 39.727584


In [1130]:
# Include rows that miss both house numbers
short_true1 = short_true[(short_true.saddno.isna()) & (short_true.PropertyAddressHouseNumber.isna())]
full_true1 = full_true[(full_true.saddno.isna()) & (full_true.PropertyAddressHouseNumber.isna())]

config_name = 'config_legal'

# LOAD the config  using jupyter %run magic ; Note the parameter escape {}
# i.e. execute variable assignment: config = ....
%run {config_name}.py

legal_map = KeplerGl(height=600, data={
    'short_false' : short_false,
    'short_true' : short_true1,
    'full_false' : full_false,
    'full_true' : full_true1, 
}, config = my_config)

legal_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'config': {'mapState': {'bearing': 0, 'dragRotate': False, 'isSplit': False, 'latitude': 40.4…

time: 154 ms (started: 2022-03-24 18:02:17 -04:00)


#### CONCLUSION: 
- all correct
- TODO later: for rows that miss both house numbers (i.e. dont have address at all), better use REGRID coords, because ATTOM coords have some duplicate/precision issues (short floats) 


## STEP2:Match the remainder using Parcel Number 

In [1135]:
# attom2 and regrid2: all rows that have not been legal matched!
attom2 = attom[~attom.at_id.isin(set(legal_matched.at_id))]
regrid2 = regrid[~regrid.re_id.isin(set(legal_matched.re_id))]
legal_matched.shape, attom2.shape, regrid2.shape

((14140, 210), (17910, 74), (12863, 136))

time: 55.9 ms (started: 2022-03-24 18:15:45 -04:00)


In [1136]:
# IMPORTANT: first, drop the duplicate values
regrid_parcelnumb = regrid2.drop_duplicates(subset='parcelnumb', keep=False)
regrid2.shape, regrid_parcelnumb.shape, 

attom_parcelnumb = attom2.drop_duplicates(subset='ParcelNumberRaw', keep=False)
attom2.shape, attom_parcelnumb.shape, 

# MERGE
parcelnumb = attom_parcelnumb.merge(regrid_parcelnumb, left_on = 'ParcelNumberRaw', right_on = 'parcelnumb', how = 'inner')[match_cols]

# Coords should have float types
parcelnumb[['lat', 'lon', 'PropertyLatitude', 'PropertyLongitude',]] = parcelnumb[['lat', 'lon', 'PropertyLatitude', 'PropertyLongitude',]].astype(float)

parcelnumb.shape, parcelnumb.shape[0] == parcelnumb.at_id.shape[0] ==  parcelnumb.re_id.unique().shape[0] ==  parcelnumb.ParcelNumberRaw.unique().shape[0]


((3669, 18), True)

time: 150 ms (started: 2022-03-24 18:15:46 -04:00)


In [1138]:
%run {config_name}.py
parcelnumb_map = KeplerGl(height=600, data={
    'full_true' : parcelnumb, 
}, config = my_config)

parcelnumb_map


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'config': {'mapState': {'bearing': 0, 'dragRotate': False, 'isSplit': False, 'latitude': 40.4…

time: 146 ms (started: 2022-03-24 18:17:16 -04:00)


In [1139]:
__config = parcelnumb_map.config
config_name = 'config_parcelnumb'

# SAVE: pretty-write kepler map config to file
with open(f"{config_name}.py", 'w') as file:
    file.write(f"my_config = {pprint.pformat(__config)}")

time: 4.73 ms (started: 2022-03-24 18:22:32 -04:00)


In [1140]:
# No house number

parcelnumb1 = parcelnumb[(parcelnumb.saddno.isna()) & (parcelnumb.PropertyAddressHouseNumber.isna())]

%run {config_name}.py
parcelnumb_map1 = KeplerGl(height=500, data={
    'full_true' : parcelnumb1, 
}, config = my_config)

parcelnumb_map1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'config': {'mapState': {'bearing': 0, 'dragRotate': False, 'isSplit': False, 'latitude': 39.5…

time: 48.9 ms (started: 2022-03-24 18:22:57 -04:00)


#### CONCLUSION: 
- all correct
- TODO later: for rows that miss both house numbers (i.e. dont have address at all), better use REGRID coords, because ATTOM coords have some duplicate/precision issues (short floats); and REGRID coords are mapped to parcels!


## STEP3: fuzzy match the remaining 

In [ ]:
match_cols_re = [
're_id', 'legaldesc', 'parcelnumb', 'address', 
'owner', 'owner2', 
'lat', 'lon','saddno', 
]

match_cols_at = [
'at_id', 'LegalDescription','ParcelNumberRaw', 'PropertyAddressFull', 
'PartyOwner1NameFull', 'PartyOwner2NameFull',
'PropertyLatitude', 'PropertyLongitude', 'PropertyAddressHouseNumber',
]

In [1157]:
# attom3 and regrid3: 
# - exclude matched LegalDesc and ParcelNumber
attom3 = attom2[~attom2.at_id.isin(set(parcelnumb.at_id))]
regrid3 = regrid2[~regrid2.re_id.isin(set(parcelnumb.re_id))]

attom.shape[0] - legal_matched.shape[0] - parcelnumb.shape[0] - attom3.shape[0]
regrid.shape[0] - legal_matched.shape[0] - parcelnumb.shape[0] - regrid3.shape[0]

((27003, 136),
 (12863, 136),
 (9194, 136),
 (32050, 74),
 (17910, 74),
 (14241, 74))

time: 34 ms (started: 2022-03-24 18:38:17 -04:00)


In [1159]:
regrid3[match_cols_re].count(), attom3[match_cols_at].count()

(re_id         9194
 legaldesc     9194
 parcelnumb    9194
 address         23
 owner         9058
 owner2           4
 lat           9194
 lon           9194
 saddno          18
 dtype: int64,
 at_id                         14241
 LegalDescription              14241
 ParcelNumberRaw               14241
 PropertyAddressFull            7513
 PartyOwner1NameFull           14241
 PartyOwner2NameFull            4633
 PropertyLatitude              14241
 PropertyLongitude             14241
 PropertyAddressHouseNumber     7042
 dtype: int64)

time: 18.1 ms (started: 2022-03-24 18:42:00 -04:00)


In [ ]:
## TODO: combine with duplicate, then
# TODO: then calculate TOTAL_SAMENESS score: weighted [ratios?] average of:
- legaldesc & parcelnumb & owner: fuzzywuzzy similarity (weighted by length [?])
- lat+lon: distance (heavier weight?)
?

In [ ]:
fuzz.token_set_ratio (TSeR) is similar to fuzz.token_sort_ratio (TSoR), except it ignores duplicated words

For comparing full names to emails, fuzz.partial_ratio (PR) is the way to go.
For addresses, fuzz.token_set_ratio (TSeR) is the scorer that performed best


In [1348]:
fuzz_scorers = [
    fuzz.ratio,
# fuzz.partial_ratio,
fuzz.token_set_ratio,
# fuzz.token_sort_ratio,
# fuzz.partial_token_set_ratio,
# fuzz.partial_token_sort_ratio,
fuzz.WRatio,
fuzz.QRatio,
# fuzz.UWRatio,
# fuzz.UQRatio,
]

time: 457 µs (started: 2022-03-26 16:09:57 -04:00)


In [ ]:
# TODO: divide into short and long df: legaldesc/parcelnumb 

In [1340]:
# should not attempt to fuzzy-match duplicate values
regrid_legal_uniq = regrid3.drop_duplicates(subset='legaldesc', keep=False)
# some unique values (10 of them) are None/uselessly short
regrid_legal_uniq = regrid_legal_uniq[regrid_legal_uniq.legaldesc.str.len() > 10]

attom_legal_uniq = attom3.drop_duplicates(subset='LegalDescription', keep=False)
attom_legal_uniq = attom_legal_uniq[attom_legal_uniq.LegalDescription.str.len() > 10]
regrid3.shape, regrid_legal_uniq.shape, attom3.shape, attom_legal_uniq.shape

((9194, 136), (8968, 136), (14241, 74), (13870, 74))

time: 61.6 ms (started: 2022-03-26 16:00:15 -04:00)


In [1339]:
# Most REGRID parcelnumb are hopelessly short
regrid3.shape, regrid3[regrid3.parcelnumb.str.len() > 8].shape # ((9194, 136), (51, 136))
# while ATTOM ParcelNumberRaw are 100% full-form
attom3.shape, attom3[attom3.ParcelNumberRaw.str.len() > 8].shape # ((14241, 74), (14241, 74))

### Conclusion: cannot fuzzy match the non-exact parcel numbers

((9194, 136), (51, 136))

time: 6.82 ms (started: 2022-03-26 15:58:18 -04:00)


In [1354]:
regrid_legal_uniq.shape, attom_legal_uniq.shape

((8968, 136), (13870, 74))

time: 2.07 ms (started: 2022-03-26 16:23:49 -04:00)


In [1349]:
data = []
index = []
for i, c in zip(regrid_legal_uniq.legaldesc, range(200)):
    index.append(i)
    row = []
    for scorer in fuzz_scorers:
        can, sim = process.extract( i, attom_legal_uniq.LegalDescription, limit=1, scorer = scorer)[0][:2]
        row.append(can)
        row.append(sim)
    data.append(row)

df_legaldesc = pd.DataFrame(data, index= index)

# 8 min to fuzzy match: 8968 regrid_legal_uniq x 13870 attom_legal_uniq (x 4 different scorers)


time: 8min 54s (started: 2022-03-26 16:10:18 -04:00)


In [1375]:
df_legaldesc = df_legaldesc[['ratio', 'token_set_ratio', 'WRatio', 'QRatio']].mode(axis=1).sample(10).rename(columns={0: 'LegalDescMajorityGuess'})[['LegalDescMajorityGuess']]

time: 49.5 ms (started: 2022-03-26 16:39:06 -04:00)


In [ ]:
# TODO HERE NOW: kepler visual: fuzzy-matched legaldesc: are the guesses close?

In [1385]:
regrid_legal_uniq.set_index('legaldesc').join(df_legaldesc).LegalDescMajorityGuess.sample(5)

legaldesc
LOTS-1-5-BLK-9-&-W-1/2-VAC-ALLEY-HUFF'S-ADD-TO-TOWN-OF-QUITMAN-EXC-PUB-RD                                                                    NaN
08-63-35-COM-578-W-OF-SE-COR-SW-TH-N-84-NW-286-S-286-E-TO-POB-NOD-CO-EXC-PUB-RD-R/W                                                          NaN
COM-CTR-SEC-29-TH-N-650.04',-E-29.45',-TO-POB,-TH-E-37.66',-NE-115.39'-SE-69.24'-E-94'-S-132.6'-W-256'-N-133-TO-POB-NOD-CO-EXC-PUB-RD-R/W    NaN
07-65-35-W-1/2-SW-EXC-PUB-RD-R/W                                                                                                             NaN
05-65-36-COM-AT-PT-1155-N-OF-CENTER-SEC-5-TH-N-1480-TH-E-1855-TH-S-1010-TH-W-33-TH-W420                                                      NaN
Name: LegalDescMajorityGuess, dtype: object

time: 29.8 ms (started: 2022-03-26 16:40:01 -04:00)


,LegalDescMajorityGuess
34-64-34-W-1/2-NE-1/4-LYING-S-OF-CO-RD-34-64-34-&-N-11-AC.-OF-NW-1/4-SE-1/4-NOD.-CO.-EXC.-PUB,34-64-34-W-12-NE-14-LYING-S-OF-CO-RD-34-64-34-...
23-66-35-E1/2-NE-&-NE-SE-&-E-3/8-SW-NE-EXC-PUB-RD-R/W,23-66-35-E12-NE-&-NE-SE-&-E-38-SW-NE-EXC-PUB-R...
30-64-34-W1/2-W1/2-NW-EXC-PUB-RD-R/W,30-64-34-W12-W12-NW-EXC-PUB-RD-RW
"COMM-AT-W1/4-CORN-SEC-22-THEN-N799.94',-W25.72'-TO-POB,-N525.44',-E115.53',-SW257.18',-SW270.03'-TO-POB","COMM-AT-SW-CORN-SEC-27-THEN-N349.23'-TO-POB,-N..."
WALLIS-1ST-ADD-LOTS-1-&-2-BLK-1,"WALLIS-1ST-ADD-LOTS-1-&-2-BLK-1-PICKERING,-MO"
"RUSSELL'S-ADD-LOTS-5--8-INC-BLK-2-ELMO,-MO","RUSSELLS-ADD-LOTS-5--8-INC-BLK-2-ELMO,-MO"
O.T.-HOPKINS-BOUNDED-BY-THOMPSON-STR-ON-S-&-RR-ON-2ND.-BLK-39,O.T.-HOPKINS-BOUNDED-BY-THOMPSON-STR-ON-S-&-RR...
18-66-34-SW-NW-EXC-PUB-RD-R/W,18-66-34-SW-NW-EXC-PUB-RD-RW
00-11-00-LOTS-1-&-2-BLK-11-O.T-BURL.-JCT.,LOTS-1-&-2-BLK-11-O.T-BURL.-JCT.
"HAWKS-1ST-ADD-&-MISC-TR-COM-AT-NE-COR-LOT-2,-BLOCK-5,-TH-S-149',-W-270',-NE-149',-E-261'-TO-POB-NOD.-CO.,-MO.","HAWKS-1ST-ADD-&-MISC-TR-COM-AT-NE-COR-LOT-2,-B..."


time: 52.5 ms (started: 2022-03-26 16:38:00 -04:00)


In [1363]:
df_legaldesc = df_legaldesc.rename(columns={
    0: 'ratio', 2: 'token_set_ratio',  4: 'WRatio', 6: 'QRatio'})
df_legaldesc.sample(8)

,ratio,1,token_set_ratio,3,WRatio,5,QRatio,7
25-66-38-E16AC-OF-NE-NE,25-66-38-E16AC-OF-NE-NE-NOD-CO-EXC-PUB-RD-RW,69,25-66-38-E16AC-OF-NE-NE-NOD-CO-EXC-PUB-RD-RW,100,25-66-38-E16AC-OF-NE-NE-NOD-CO-EXC-PUB-RD-RW,90,25-66-38-E16AC-OF-NE-NE-NOD-CO-EXC-PUB-RD-RW,69
"POB-IS-383'N-&-24'E-OF-SW-COR-S19-66-36,-N.79'-E.138'-S,79'-W-138-TO-POB","POB-IS-383N-&-24E-OF-SW-COR-S19-66-36,-N.79,-E...",92,"POB-IS-383N-&-24E-OF-SW-COR-S19-66-36,-N.79,-E...",93,"POB-IS-383N-&-24E-OF-SW-COR-S19-66-36,-N.79,-E...",92,"POB-IS-383N-&-24E-OF-SW-COR-S19-66-36,-N.79,-E...",92
00-11-00-LOTS-1-&-2-BLK-11-O.T-BURL.-JCT.,LOTS-1-&-2-BLK-11-O.T-BURL.-JCT.,87,LOTS-1-&-2-BLK-11-O.T-BURL.-JCT.,100,LOTS-1-&-2-BLK-11-O.T-BURL.-JCT.,95,LOTS-1-&-2-BLK-11-O.T-BURL.-JCT.,87
"SCOTT'S-ADD-LOTS-2,-3-&-4-BLK-4-ELMO","SCOTTS-ADD-LOTS-2,-3-&-4-BLK-4-ELMO",99,"SCOTTS-ADD-LOTS-2,-3-&-4-BLK-4-ELMO",95,"SCOTTS-ADD-LOTS-2,-3-&-4-BLK-4-ELMO",99,"SCOTTS-ADD-LOTS-2,-3-&-4-BLK-4-ELMO",99
13-64-35-LOT-5-MOZINGO-GREEN-ACRES,1-64-35-RE-PLAT-MOZINGO-LAKE-ESTATES-LOT-9,63,13-64-35-LOT-5-MOZINGO-GREEN-ACRES-COM-NW-COR-...,100,13-64-35-LOT-5-MOZINGO-GREEN-ACRES-COM-NW-COR-...,90,1-64-35-RE-PLAT-MOZINGO-LAKE-ESTATES-LOT-9,63
18-63-33-NE-NE-N-OF-RR-EXC-PUB-RD-R/W,18-63-33-NE-NE-N-OF-RR-EXC-PUB-RD-RW,99,18-63-33-NE-NE-N-OF-RR-EXC-PUB-RD-RW,99,18-63-33-NE-NE-N-OF-RR-EXC-PUB-RD-RW,99,18-63-33-NE-NE-N-OF-RR-EXC-PUB-RD-RW,99
SE-1/4-SEC-15-LYING-E-OF-CL-ABANDONED-RR-&-W-OF-COUNTY-RD-EXC-N-1/2-N1/2-SE-1/4,"S-14-SE-14,-4-65-37,-LYING-N-&-E-OF-CREEK,-NOD...",56,E-1/2-SE-1/4-SEC-12,90,22-62-33-SW-14-NW-14-EXC-PUB-RD-RW,86,"S-14-SE-14,-4-65-37,-LYING-N-&-E-OF-CREEK,-NOD...",56
RR-R/W-COM-180-W-OF-NE-COR-LOT-1-BLK-54-O.T.-HOPKINS-SW-140'-TH-W-30'-TH-N-120'-TH,RR-RW-COM-180-W-OF-NE-COR-LOT-1-BLK-54-O.T.-HO...,86,RR-RW-COM-180-W-OF-NE-COR-LOT-1-BLK-54-O.T.-HO...,99,RR-RW-COM-180-W-OF-NE-COR-LOT-1-BLK-54-O.T.-HO...,94,RR-RW-COM-180-W-OF-NE-COR-LOT-1-BLK-54-O.T.-HO...,86


time: 7.23 ms (started: 2022-03-26 16:31:06 -04:00)


In [ ]:
# TODO : VISUAL inspect scorers, and select (multiple scorers for same attribute, then take majority? what if a scorer selection contain repeats?)
## and weight multiple attributes

In [1033]:
# fuzzy match score magic cell:  ! pip install fuzzywuzzy[speedup
# https://medium.com/analytics-vidhya/matching-messy-pandas-columns-with-fuzzywuzzy-4adda6c7994f
# If doesn't work: install using CLI: pip install fuzzywuzzy and then install python-levenshtein like this: pip install python-Levenshtein (in order to speed the matching process).
from fuzzywuzzy import process, fuzz
import matplotlib.pyplot as plt


((26992, 139), (25858, 139), (32050, 77), (30778, 77), (14106, 216), True)

time: 644 ms (started: 2022-03-24 13:43:12 -04:00)


In [18]:
regrid_cols = ['address', 'city', 'county', 'dpv_codes', 'dpv_notes', 'dpv_status', 'dpv_type', 'geometry', 'gisacre', 'lat', 'lbcs_activity', 'lbcs_function', 'lbcs_ownership', 'lbcs_site', 'lbcs_structure', 'legaldesc', 'll_bldg_count', 'll_bldg_footprint_sqft', 'll_gisacre', 'll_uuid', 'lon', 'owner', 'owner2', 'parcelnumb', 'rdi', 'saddno', 'saddpref', 'saddstr', 'saddstsuf', 'saddsttyp', 'saledate', 'saleprice', 'scity', 'sunit', 'szip', 'usps_vacancy']

time: 437 µs (started: 2022-04-08 09:54:01 -04:00)


In [19]:
main_df = gp.read_file('/home/nhat/regrid-bucket/tx_kenedy.json')[regrid_cols]

time: 281 ms (started: 2022-04-08 09:54:13 -04:00)


In [8]:
import pandas as pd
attom_cols = [
'[ATTOM ID]', 
'SitusStateCode', # DONT NEED 
'AreaBuilding', 'AreaBuildingDefinitionCode', 'AreaLotSF', 'BuildingsCount', 'CompanyFlag', 
'ContactOwnerMailAddressFull', 'ContactOwnerMailAddressInfoFormat', 'GeoQuality', 
'LegalDescription', 'OwnerTypeDescription1', 'ParcelNumberRaw', 'PartyOwner1NameFull', 'PartyOwner2NameFull', 
'PartyOwner3NameFull', # DONT NEED
'PropertyAddressCity', 'PropertyAddressFull', 'PropertyAddressHouseNumber', 
'PropertyAddressState',  # DONT NEED
'PropertyAddressStreetDirection', 'PropertyAddressStreetName', 
'PropertyAddressStreetPostDirection', 'PropertyAddressStreetSuffix', 'PropertyAddressUnitPrefix', 'PropertyAddressUnitValue', 
'PropertyAddressZIP', 'PropertyAddressZIP4', 'PropertyLatitude', 'PropertyLongitude', 
'PropertyUseGroup', 'PropertyUseStandardized', 'RoomsCount', 'StatusOwnerOccupiedFlag', 'StoriesCount', 'UnitsCount', 
'TaxMarketValueYear', 'TaxMarketValueImprovements', 'YearBuilt']
 
df = pd.read_json('attom/data/parsed_attom_DC.json',
                 orient = 'values', lines = True)
df.columns = attom_cols + ['zip_order', 'row_order']

In [10]:
df[['SitusStateCode', 'PropertyAddressState']]

,SitusStateCode,PropertyAddressState
0,DC,DC
1,DC,DC
2,DC,DC
3,DC,DC
4,DC,DC
...,...,...
213074,DC,DC
213075,DC,DC
213076,DC,DC
213077,DC,DC
